# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
import pandas as pd
import os
import boto3
import sagemaker

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [3]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'plagiarism_data'

# upload all data to S3
train_path  = sagemaker_session.upload_data(os.path.join(data_dir, 'train.csv'), bucket=bucket, key_prefix=prefix)
test_path   = sagemaker_session.upload_data(os.path.join(data_dir, 'test.csv'),  bucket=bucket, key_prefix=prefix)

### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    if data_dir in obj.key:
        empty_check.append(obj.key)
        print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

plagiarism_data/output/sagemaker-xgboost-2020-06-13-08-55-56-078/output/model.tar.gz
plagiarism_data/output/sagemaker-xgboost-2020-06-13-09-40-10-735/output/model.tar.gz
plagiarism_data/output/xgboost-2020-06-13-09-29-25-876/output/model.tar.gz
plagiarism_data/test.csv
plagiarism_data/train.csv
plagiarism_data/xgb_test.csv
Test passed!


---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## Use built-in algorithm XGBoost

As a sort of baseline, I decided to train use the XGBoost built-in classification algorithm. I choose XGBoost because it's a popular algorithm that performs well against all sorts of data. As this is a baseline, I'm using the hyperparameters used in the sentiment analysis example without further tuning.

In [5]:
from sagemaker.amazon.amazon_estimator import get_image_uri

# Use the latest xgboost algorithm.
container = get_image_uri(sagemaker_session.boto_region_name,
                          'xgboost',
                          repo_version='0.90-2')

xgb_output_path = 's3://{}/{}/output'.format(bucket, prefix)

xgb = sagemaker.estimator.Estimator(container,
                                    role,
                                    train_instance_count=1,
                                    train_instance_type='ml.m4.xlarge', # M family recommended
                                    output_path=xgb_output_path,
                                    sagemaker_session=sagemaker_session)

xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        num_round=500)

### Training

Start the actual training of the model.

In [6]:
%%time

s3_input_train = sagemaker.s3_input(s3_data=train_path, content_type='csv')
xgb.fit({'train': s3_input_train})

2020-06-13 12:16:14 Starting - Starting the training job...
2020-06-13 12:16:16 Starting - Launching requested ML instances.........
2020-06-13 12:17:48 Starting - Preparing the instances for training...
2020-06-13 12:18:44 Downloading - Downloading input data...
2020-06-13 12:19:03 Training - Downloading the training image...
2020-06-13 12:19:45 Uploading - Uploading generated training model.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[12:19:42] 70x4 matrix with 280 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Single node

### Testing the model

The test data was uploaded onto S3 in a format that cannot be used for predictions (the labels are included). In order to be able to use it to test the model, I need to upload the test data without the labels.

In [7]:
# Read the test data.
test_df = pd.read_csv(os.path.join(data_dir, 'test.csv'))

# Seperate labels and features
test_y = test_df.iloc[:,0]
test_X = test_df.iloc[:, 1:]

# Dump the test features and upload to S3.
xgb_test_file = 'xgb_test.csv'
pd.DataFrame(test_X).to_csv(os.path.join(data_dir, xgb_test_file), header=False, index=False)
test_path = sagemaker_session.upload_data(os.path.join(data_dir, xgb_test_file),  bucket=bucket, key_prefix=prefix)

Create a batch transformer and perform the predictions on the test data.

In [8]:
xgb_transformer = xgb.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')

In [9]:
%%time

xgb_transformer.transform(test_path, content_type='text/csv', split_type='Line')
xgb_transformer.wait()

.......................[2020-06-13:12:24:08:INFO] No GPUs detected (normal if no gpus installed)
[2020-06-13:12:24:08:INFO] No GPUs detected (normal if no gpus installed)
[2020-06-13:12:24:08:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;

worker_rlimit_nofile 4096;

events {
  worker_connections 2048;
}

http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;

  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }

  server {
    listen 8080 deferred;
    client_max_body_size 0;

    keepalive_timeout 3;

    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }

    location / {
      return 404 "{}";
    }

  }
}


[2020-06-13 12:24:08 +0000] [19] [INFO] Starting g

In [10]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

download: s3://sagemaker-eu-west-1-019518462631/sagemaker-xgboost-2020-06-13-12-20-27-217/xgb_test.csv.out to plagiarism_data/xgb_test.csv.out


And finally check the accuracy.

In [11]:
from sklearn.metrics import accuracy_score

predictions = pd.read_csv(os.path.join(data_dir, xgb_test_file+'.out'), header=None)
predictions = [round(num) for num in predictions.squeeze().values]

accuracy_score(test_y, predictions)

0.9583333333333334

Baseline accuracy is **95.83%**, which is obviously excellent.

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

---

### Classifier algorithm choice

**For the exercise I've chosen to use a custom Scikit learn classifier based on Support Vector Machines: `sklearn.svm.SVC` (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).**

**From the Udacity course 'Machine Learning - Introduction', I remember that SVM is a powerfull algorithm for classification, which aims not only to find the best possible boundary between the classes, but also maximize the margin (distance between that boundary and the training data points for both classes). SVM supports a non-linear boundary, which means it can handle more complicated data sets. The risk with non-linear boundaries is that they can result in overfitting. However the margin maximization helps SVM to avoid overfitting even in classification problems with a small data set.
A disadvantage is that training SVM with a large dataset will be time consuming. However the training set only contains 75 items, so that will not be an issue.**

SVM has a set of hyperparameters:

* C: balances error between misclassification (large value) and margin between boundary and datapoints (small value).
* kernel: SVM supports multiple kernels, which offer different projection functions for the data points in order to find a better and more complicated boundary.

---

In [13]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd

from sklearn.externals import joblib

## TODO: Import any additional libraries you need to define a model
from sklearn.svm import SVC


# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")
    
    return model


## TODO: Complete the main code
if __name__ == '__main__':
    
    # All of the model parameters and training parameters are sent as arguments
    # when this script is executed, during a training job
    
    # Here we set up an argument parser to easily access the parameters
    parser = argparse.ArgumentParser()

    # SageMaker parameters, like the directories for training data and saving models; set automatically
    # D

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [14]:
from sagemaker.sklearn.estimator import SKLearn

# pass defaults from sklearn documentation.
hyperparameters = {
    'kernel': 'rbf',
    'c': 1.0
}

estimator = SKLearn(entry_point='train.py',
                    source_dir='source_sklearn',
                    role=role,
                    sagemaker_session=sagemaker_session,
                    train_instance_count=1,
                    train_instance_type='ml.m4.xlarge',
                    hyperparameters=hyperparameters)


## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [15]:
%%time

# Train your estimator on S3 training data
estimator.fit({'train': train_path})

2020-06-13 12:41:56 Starting - Starting the training job...
2020-06-13 12:41:57 Starting - Launching requested ML instances.........
2020-06-13 12:43:30 Starting - Preparing the instances for training...
2020-06-13 12:44:13 Downloading - Downloading input data...
2020-06-13 12:44:57 Training - Training image download completed. Training in progress..2020-06-13 12:44:57,907 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-06-13 12:44:57,910 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-13 12:44:57,928 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-06-13 12:44:58,186 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-06-13 12:44:58,186 sagemaker-containers INFO     Generating setup.cfg
2020-06-13 12:44:58,186 sagemaker-containers INFO     Generating MANIFEST.in
2020-06-13 12:44:58,186 sagemaker-containers INFO     Installing mo

## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [16]:
%%time

# deploy your model to create a predictor
predictor = estimator.deploy(instance_type='ml.t2.medium',
                             initial_instance_count=1)


---------------!CPU times: user 249 ms, sys: 22 ms, total: 271 ms
Wall time: 7min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [18]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [28]:
# Second: calculate the test accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, test_y_preds)

print(accuracy)

## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

0.96

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


In [29]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(test_y, test_y_preds).ravel()

recall = tp / (tp + fn)
precision = tp / (tp + fp)
accuracy = (tp + tn) / (tp + fp + tn + fn)

print(pd.crosstab(test_y, test_y_preds, rownames=['actuals'], colnames=['predictions']))
print("\n{:<11} {:.3f}".format('Recall:', recall))
print("{:<11} {:.3f}".format('Precision:', precision))
print("{:<11} {:.3f}".format('Accuracy:', accuracy))

predictions   0   1
actuals            
0            10   0
1             1  14

Recall:     0.933
Precision:  1.000
Accuracy:   0.960


In [30]:
df = pd.concat([pd.DataFrame(test_x), pd.DataFrame(test_y), pd.DataFrame(test_y_preds)], axis=1)
df.columns=['c_1', 'c_13', 'c_19', 'lcs_word', 'class', 'predicted']
df

,c_1,c_13,c_19,lcs_word,class,predicted
0,1.000000,0.745946,0.603352,0.820755,1,1
1,0.765306,0.361702,0.286232,0.621711,1,1
2,0.884444,0.004695,0.000000,0.597458,1,1
3,0.619048,0.000000,0.000000,0.427835,1,1
4,0.920000,0.079365,0.000000,0.775000,1,1
5,0.992674,0.942529,0.917647,0.993056,1,1
6,0.412698,0.000000,0.000000,0.346667,0,0
7,0.462687,0.000000,0.000000,0.189320,0,0
8,0.581152,0.000000,0.000000,0.247423,0,0
9,0.584211,0.000000,0.000000,0.294416,0,0


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

**Answer**: 

The model achieved an accuracy of 96%.

Of the 25 items in the test data, 24 were classified correctly. 1 plagiarized text was classified as non-plagiarized (false negative).

When comparing the features in the table above, the values of the features of the misclassified answer (index 12) are always on the low end compared to the features of the other plagiarized answer texts. Lower values means that the answer text has less commonalities with the source text.

* c_1 (1 ngrams) is 0.6197.
* c_13 (13 ngrams) and c_19 (19 ngrams) are 0
* lcs_words is 0.34

After going back to the data set, the misclassified answer text was actually in file g2pB_taskc.txt, which is categorized as heavy modified (the text was heavily modified (e.g. changing words) compared to the original source text).

### Question 2: How did you decide on the type of model to use? 

**Answer**:

As explained above (Classifier algorithm choice), I choose SVM because it is a powerfull algorithm to use on a classification problem. It supports non-linear boundaries (using poly and rbf kernels). SVM works well with smaller datasets with many features.

The dataset for this particular project is very small and has few features, which makes it a good fit to use SVM.

(Source: https://elitedatascience.com/machine-learning-algorithms)

----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [26]:
# uncomment and fill in the line below!
predictor.delete_endpoint()


### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [ ]:
# deleting bucket, uncomment lines below

# bucket_to_delete = boto3.resource('s3').Bucket(bucket)
# bucket_to_delete.objects.all().delete()

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!